In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
# Tensorflow gets its own cell. There are issues with it and my current processor.
import tensorflow as tf

2023-02-20 15:56:55.118371: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

In [5]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [6]:
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
aaa = application_df['SPECIAL_CONSIDERATIONS'].value_counts()
aaa

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [8]:
bbb = application_df['STATUS'].value_counts()
bbb

1    34294
0        5
Name: STATUS, dtype: int64

In [9]:
# After some Data Exploration, it appears these columns would be of minimal value. I experiemented with
# dropping them also, but it appeared to actually hurt the accuracy.

# application_df = application_df.drop(columns=["SPECIAL_CONSIDERATIONS", "STATUS"])

In [10]:
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [11]:
#List for cell below
overten = []
overten = application_counts.loc[application_counts < 10].index.tolist()
overten

['T25', 'T14', 'T29', 'T15', 'T17']

In [12]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = overten
#['T17','T15','T29','T14','T25']
#['T9','T13','T12', 'T2','T25','T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [13]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [14]:
# You may find it helpful to look at CLASSIFICATION value counts >1
sf_df = application_df['CLASSIFICATION'].value_counts()
sf_df = sf_df.loc[sf_df <= 3]
sf_df

C1260    3
C2710    3
C0       3
C3200    2
C1234    2
C1246    2
C1267    2
C1256    2
C2190    1
C4200    1
C2600    1
C5200    1
C1370    1
C1248    1
C6100    1
C1820    1
C1900    1
C1236    1
C3700    1
C2570    1
C1580    1
C1245    1
C2500    1
C1570    1
C1283    1
C2380    1
C1732    1
C1728    1
C2170    1
C4120    1
C8210    1
C2561    1
C4500    1
C2150    1
Name: CLASSIFICATION, dtype: int64

In [15]:
#Creating a list for the next step
three_under = []
three_under = sf_df.index.tolist()
three_under

['C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [16]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = three_under
#classifications_to_replace = sf_df.loc[sf_df <= 287].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Other       45
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C4100        6
C1720        6
C2400        6
C1257        5
C1600        5
Name: CLASSIFICATION, dtype: int64

In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
application_with_dummies_df = pd.get_dummies(application_df)

In [18]:
# Split our preprocessed data into our features and target arrays
X = application_with_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_with_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 10
# hidden_nodes_layer4 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))
# Fourth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               8000      
                                                                 
 dense_1 (Dense)             (None, 40)                4040      
                                                                 
 dense_2 (Dense)             (None, 10)                410       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 12,461
Trainable params: 12,461
Non-trainable params: 0
_________________________________________________________________


2023-02-20 15:56:56.862508: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=110)

Epoch 1/110
804/804 [==============================] - 1s 611us/step - loss: 0.5954 - accuracy: 0.7020
Epoch 2/110
804/804 [==============================] - 0s 599us/step - loss: 0.5689 - accuracy: 0.7264
Epoch 3/110
804/804 [==============================] - 0s 609us/step - loss: 0.5612 - accuracy: 0.7271
Epoch 4/110
804/804 [==============================] - 0s 601us/step - loss: 0.5560 - accuracy: 0.7281
Epoch 5/110
804/804 [==============================] - 0s 599us/step - loss: 0.5539 - accuracy: 0.7285
Epoch 6/110
804/804 [==============================] - 0s 594us/step - loss: 0.5528 - accuracy: 0.7290
Epoch 7/110
804/804 [==============================] - 0s 595us/step - loss: 0.5517 - accuracy: 0.7314
Epoch 8/110
804/804 [==============================] - 0s 593us/step - loss: 0.5507 - accuracy: 0.7310
Epoch 9/110
804/804 [==============================] - 0s 592us/step - loss: 0.5494 - accuracy: 0.7313
Epoch 10/110
804/804 [==============================] - 0s 596us/step - l

804/804 [==============================] - 0s 595us/step - loss: 0.5304 - accuracy: 0.7402
Epoch 80/110
804/804 [==============================] - 0s 587us/step - loss: 0.5302 - accuracy: 0.7409
Epoch 81/110
804/804 [==============================] - 0s 590us/step - loss: 0.5303 - accuracy: 0.7409
Epoch 82/110
804/804 [==============================] - 0s 597us/step - loss: 0.5299 - accuracy: 0.7399
Epoch 83/110
804/804 [==============================] - 1s 622us/step - loss: 0.5297 - accuracy: 0.7398
Epoch 84/110
804/804 [==============================] - 0s 607us/step - loss: 0.5298 - accuracy: 0.7411
Epoch 85/110
804/804 [==============================] - 0s 591us/step - loss: 0.5297 - accuracy: 0.7420
Epoch 86/110
804/804 [==============================] - 0s 589us/step - loss: 0.5294 - accuracy: 0.7407
Epoch 87/110
804/804 [==============================] - 0s 591us/step - loss: 0.5293 - accuracy: 0.7410
Epoch 88/110
804/804 [==============================] - 0s 588us/step - loss:

In [23]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5601 - accuracy: 0.7308 - 159ms/epoch - 592us/step
Loss: 0.5600593686103821, Accuracy: 0.7308454513549805


In [24]:
nn.save("AlphabetSoupCharity_Optimization.h5")

# Summary

### Purpose
To ascertain applicants with the best chance of success

#### The target column for this excercise was the "Is-Successful" column. The "EIN" and "Name" columns were dropped and the rest of the columns are considered features.



##### This exercise was performed times and could not reach 75 percent accuracy. The highest accuracy was 73.08%.

![accuracy image](Resources/screen_one.png "accuracy image")

The best results were generated by creating the largest data set possible by dropping smaller sets of rows.  The best results came with sigmoid as opposed to relu functions. Again this tied into the idea of a denser data sets leading to greater success.  Even dropping columns that appeared to have little relevant data dropped the accuracy score.

I feel there could be a very fine line between the number of nodes necessary to process the data enough before overfitting happens. That was the challenge of this particular assignment.

#### Conclusion

While 34000 organization is a large number for a human to process, more information or even a larger count of organizations is necessary to produce a reliably consistent model.